<h2>Extract Frames from video</h2>

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from pathlib import Path
vi = "./vid/v1.mp4"
print(vi)
vi = Path(vi)
print(vi)
print(vi.stem)

directory = Path(f"./data_3rd_{vi.stem}")
directory.mkdir(parents=True, exist_ok=True)

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from pathlib import Path

vi = "./vid/v1.mp4"
vi = Path(vi)
frames_directory = f"./data_3rd_{vi.stem}"

cap = cv2.VideoCapture(vi)

directory = Path(frames_directory)
directory.mkdir(parents=True, exist_ok=True)

frame_no = 300

frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

req_frames = [i for i in range(int(frame_count)) if i%1800 == 0 and i!=0]
r = []
for i in req_frames:
    a,b,c = i-2,i-1,i
    r.append(a)
    r.append(b)
    r.append(c)
# print(req_frames,len(req_frames))
print(r,len(r))
frames=[]

#Save Frames in order
for j,i in enumerate(r,start=1):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    res, frame = cap.read()
    if res:
        name=f'./data_3rd_{vi.stem}/frame{j}_{str(i)}.jpg'
        # frames.append(name)
        frames.append(Path(name))
        # print('creating'+name)
        cv2.imwrite(name,frame)

cap.release()

print(frames,len(frames))

<h2>Emotion detection function</h2>

In [ ]:
from deepface import DeepFace

def detect_emotion_mtcnn(path,backend="mtcnn"):
  try:
      analysis = DeepFace.analyze(
          img_path=path,
          actions=["emotion"],
          # detector_backend="facenet" #(facenet)
          detector_backend=backend,#(MTCNN)
          enforce_detection=False,
          expand_percentage=30
      )

      # print("Emotion Analysis:")
      # print(analysis[0]["emotion"])
      # print(f"Dominant Emotion: {analysis[0]['dominant_emotion']}")
      
      next_dominant_emotion=analysis[0]['emotion']

      return analysis[0]['dominant_emotion']

  except Exception as e:
      # print(f"An error occurred: {e}")
      return None

<h2>Eye Gaze function</h2>

In [ ]:
# !pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp


# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=100, refine_landmarks=True, min_detection_confidence=0.2, min_tracking_confidence=0.3)

def eye_aspect_ratio(eye_landmarks, landmarks):
    A = np.linalg.norm(np.array(landmarks[1]) - np.array(landmarks[5]))
    B = np.linalg.norm(np.array(landmarks[2]) - np.array(landmarks[4]))
    C = np.linalg.norm(np.array(landmarks[0]) - np.array(landmarks[3]))
    ear = (A + B) / (2.0 * C)
    return ear


def detect_eye_pose(image):
    """
    Detects eye pose based on the position of the iris.
    Args:
        image: Input image
    Returns:
        Annotated image with eye pose directions
    """
    h, w = image.shape[:2]
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    results = face_mesh.process(rgb_image)

    if not results.multi_face_landmarks:
        # print("No face detected.")
        return "No face detected."

    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:

        # Define eye landmarks (Mediapipe indexes)
        left_eye_indexes = [33, 159, 158, 133, 153, 145]  # Example keypoints for left eye
        right_eye_indexes = [362, 380, 374, 263, 386, 385]  # Example keypoints for right eye



        # left_eye_indexes = [463, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 362]
        # right_eye_indexes = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]

        # Extract coordinates for left and right eyes
        left_eye = [(int(face_landmarks.landmark[i].x * w),
                     int(face_landmarks.landmark[i].y * h)) for i in left_eye_indexes]
        right_eye = [(int(face_landmarks.landmark[i].x * w),
                      int(face_landmarks.landmark[i].y * h)) for i in right_eye_indexes]

        l_ear = eye_aspect_ratio(left_eye_indexes,left_eye)
        r_ear = eye_aspect_ratio(right_eye_indexes,right_eye)

        EAR_THRESHOLD_S = 0.1 # threshold for closing eyes
        EAR_THRESHOLD_M = 0.18 # threshold for closing eyes
        if l_ear<=EAR_THRESHOLD_S or r_ear<=EAR_THRESHOLD_S:
            gaze_direction='LD'
        elif l_ear<=EAR_THRESHOLD_M or r_ear<=EAR_THRESHOLD_M:
            gaze_direction='LUM'
        else:
            gaze_direction='LU'

        annotated_image = cv2.resize(annotated_image, (512, 512))

        # Annotate the image
        # cv2.putText(annotated_image, gaze_direction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(annotated_image, gaze_direction, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # return gaze_direction
        # return f"{gaze_direction}, {l_ear}, {r_ear}"
        return gaze_direction

    # return annotated_image

# Load an example image
# image_path = "/content/detected_faces/face_2.jpg"
# image = cv2.imread(image_path)

# Detect eye pose
# output_image = detect_eye_pose(image)

# Display the result
# cv2_imshow(output_image)
# cv2.imshow("Eye Pose Detection", output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# !pip install mediapipe --user

<h1>Head Tilt</h1>

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os


# Camera matrix (assume focal length = width of the image for simplicity)
def get_camera_matrix(image):
    height, width = image.shape[:2]
    focal_x = width
    focal_y = height
    center = (width, height)
    # center = (width / 3, height / 5)
    return np.array([
        [focal_x, 0, center[0]],
        [0, focal_x, center[1]],
        [0, 0, 1]
    ], dtype="double")

# Define a function to classify head pose
def classify_head_pose(pitch, yaw, roll, img_c, face_c):
    # if abs(yaw) < 10 and abs(pitch) < 10:
    if abs(yaw) < 10 and abs(pitch) < 10:
        # return "Looking Straight"
        return "LS"
    elif yaw > 10:
        # return "Looking Left"
        if face_c>=img_c:
            return "LS"
        return "LL"
    elif yaw < -10:
        # return "Looking Right"
        if face_c <=img_c:
            return "LS"
        return "LR"
    elif pitch > 10:
        # return "Looking Down"
        return "LD"
    elif pitch < -10:
        # return "Looking Up"
        return "LU"
    # return "Uncertain"
    return "U"


# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh 
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True,min_detection_confidence=0.02)

def head_pose_classifier(faceid,image, img_c, face_c):

  # Load the image
  # image = cv2.imread(image_path)

  if image is None:
      raise ValueError("Image not found.")
  rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Detect face mesh
  results = face_mesh.process(rgb_image)

  if results.multi_face_landmarks:
  
      for face_landmarks in results.multi_face_landmarks:
          # Define 3D model points (generic face)
          model_points = np.array([
              (0.0, 0.0, 0.0),             # Nose tip
              (0.0, -330.0, -65.0),        # Chin
              (-225.0, 170.0, -135.0),     # Left eye left corner
              (225.0, 170.0, -135.0),      # Right eye right corner
              (-150.0, -150.0, -125.0),    # Left mouth corner
              (150.0, -150.0, -125.0)      # Right mouth corner
          ], dtype="double")

          # Extract 2D image points from Mediapipe landmarks
          landmarks = face_landmarks.landmark
          image_points = np.array([
              (landmarks[1].x * image.shape[1], landmarks[1].y * image.shape[0]),  # Nose tip
              (landmarks[152].x * image.shape[1], landmarks[152].y * image.shape[0]),  # Chin
              (landmarks[33].x * image.shape[1], landmarks[33].y * image.shape[0]),  # Left eye
              (landmarks[263].x * image.shape[1], landmarks[263].y * image.shape[0]),  # Right eye
              (landmarks[287].x * image.shape[1], landmarks[287].y * image.shape[0]),  # Left mouth corner
              (landmarks[57].x * image.shape[1], landmarks[57].y * image.shape[0])   # Right mouth corner
          ], dtype="double")

          # SolvePnP to estimate head pose
          camera_matrix = get_camera_matrix(image)
          dist_coeffs = np.zeros((4, 1))  # Assume no distortion
          success, rotation_vector, translation_vector = cv2.solvePnP(
              model_points, image_points, camera_matrix, dist_coeffs
          )

          # Convert rotation vector to Euler angles (pitch, yaw, roll)
          rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
          pose_matrix = np.hstack((rotation_matrix, translation_vector))
          _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_matrix)
          pitch, yaw, roll = euler_angles.flatten()

          # Classify head pose
          head_pose = classify_head_pose(pitch, yaw, roll, img_c, face_c)

         
          # return f"Pitch: {pitch:.2f}, Yaw: {yaw:.2f}, Roll: {roll:.2f} -> {head_pose}"
          # print(f"***{faceid} Pitch: {pitch:.2f}, Yaw: {yaw:.2f}, Roll: {roll:.2f} -> {head_pose}")
          # image = cv2.resize(image, (64, 64))

          # Annotate the image with the classification
          # cv2.putText(image, head_pose, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
          # cv2.putText(image, f"Pitch: {pitch:.2f}, Yaw: {yaw:.2f}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
          # cv2.imshow(image)
          return head_pose,pitch,yaw,roll
      return None, 0,0,0
    
  else:
      # print(f"***{faceid}")
      return None, 0,0,0

      

  # Display the image
  # cv2.imshow("Head Pose Classification", image)
  # cv2_imshow(image)
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()


# Detect, crop and save faces in frame

In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
import os


def count_faces(input_image_path,df,img, backend):
    image = cv2.imread(input_image_path)
    if image is None:
        raise ValueError("Image not found. Please check the path.")

    # Convert the image to RGB (MTCNN requires RGB input)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize the MTCNN detector
    detector = MTCNN()

    # Detect faces
    faces = detector.detect_faces(rgb_image)
    hc,wc = rgb_image.shape[:2] 
    # wc=wc//2
    # a,b,_ = rgb_image.shape
    # print(a,b)

    # f={}

    for i,face in enumerate(faces):
        x, y, w, h = face['box']
        # print("dim",x,y,w,h)
    
        cv2.rectangle(rgb_image, (x-7, y-7), (x + w+7, y + h+7), (0, 0, 255), 2)
        # Crop face  - enlarged the crop area for bettr head pose detection
        face_crop = rgb_image[y-7:y + h+7, x-7:x + w+7]
        # faces = []
        name = f"face_{i}"
        # faces.append(name)

        # face_crop = image[y:y + h, x:x + w]
        # p1 = detect_emotion_opencv(face_crop)
        p1 = detect_emotion_mtcnn(face_crop,backend)
        fc=x+w//2
        
        p2,p,yaw,r= head_pose_classifier(name,face_crop,wc//2,fc )
        p3 = detect_eye_pose(face_crop)

        d = [input_image_path,name,p1,p2,p3,x,y,w,h]
#   
        df.loc[len(df)] = d
        # f.update({name:d})
        # cv2.putText(rgb_image,f'{name}{p1} {p2} {p3}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
      # for head pose
        # cv2.putText(rgb_image,f'{name}{p2} p{p:.2f} y{yaw:.2f} r{r:.2f}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    return rgb_image


In [ ]:
import os
import pandas as pd
import time

# your code here    
# def consolidated_emotion_score()

def multimodal_counts(path,fr,backend):
    df = pd.DataFrame(columns=["Frame","Face", "Emotion","HeadTilt","EyeGaze","x","y","w","h"])
    # processed_frame = preprocess(path)print(os.listdir())
    # Every 5 minutes
    start = time.process_time()
    frame = count_faces(path,df,fr,backend)
    name=f'./processed_3rd/{fr}.jpg'
    # print(name,frame)
    cv2.imwrite(name,frame)
 
    return df, time.process_time() - start




<h1>TOPSIS</h1>

In [ ]:


def multimodal_backend(backend, num_frames):
    emotion_rank = {'neutral':6, 'happy':4, 'sad':5, 'angry':3,'surprise':4, 'disgust':2, 'fear':2}
    head_rank = {'LL':3,'LR':3,'LU':4,'LD':1,'LS':5,None:2}
    eye_rank = {'LD':2,'LUM':3,'LU':4,'No face detected.':1}
        
    weights = {"Emotion":(1/3),"HeadTilt":(1/3),"EyeGaze":(1/3)}
    print(f"----{backend}----")
    conc_indices=[]
    times=[]
    emotions=[]
    head_tilt=[]
    eyes=[]
    df_all=pd.DataFrame(columns=["Frame","Face", "Emotion","HeadTilt","EyeGaze","x","y","w","h","EmotionRank","HeadTiltRank","EyeGazeRank",
            "square_sum1",'square_sum2','square_sum3','EmotionNormalised','HeadTiltNormalised','EyeGazeNormalised',
           'a_Vi+','a_Vi-','b_Vi+','b_Vi-','c_Vi+','c_Vi-','Si+','Si-','Pi','Engaged/Disengaged','Rank'])
    for i,path in enumerate(frames[:num_frames]):
        df,t=multimodal_counts(path,f'frame{i}',backend)
        print(f">>Frame {i}")
        
        times.append(t)
        emotions.append(df['Emotion'].value_counts())
        head_tilt.append(df['HeadTilt'].value_counts())
        eyes.append(df['EyeGaze'].value_counts())
        # print(f'Concentration Index frame {i} {ci}')
    

        # df,_ = multimodal_counts('f1.jpg','frame1','mtcnn')
        df["EmotionRank"] = df["Emotion"].map(lambda x: emotion_rank[x])
        df["HeadTiltRank"] = df["HeadTilt"].map(lambda x: head_rank[x])
        df["EyeGazeRank"] = df["EyeGaze"].map(lambda x: eye_rank[x])
        
        # df['square_sum1'] = df['EmotionRank'].map(lambda x: x**2)
        df['square_sum1'] = df['EmotionRank'].map(lambda x: x**2)
        df['square_sum2'] = df['HeadTiltRank'].map(lambda x: x**2)
        df['square_sum3'] = df['EyeGazeRank'].map(lambda x: x**2)
        
        s1 = (df['square_sum1'].sum())**0.5
        s2 = (df['square_sum2'].sum())**0.5
        s3 = (df['square_sum3'].sum())**0.5
        print(s1,s2,s3)
            
        def vector_normalisation(x):
            s=0
            s+=x**2
            return (x/s**0.5)
        
        # df = df[['Frame','Face','EmotionRank','HeadTiltRank','EyeGazeRank']].copy()
        
        df["EmotionNormalised"]=df["EmotionRank"].map(lambda x: x/s1)*weights["Emotion"]
        df["HeadTiltNormalised"]=df["HeadTiltRank"].map(lambda x: x/s2)*weights["HeadTilt"]
        df["EyeGazeNormalised"]=df["EyeGazeRank"].map(lambda x: x/s3)*weights["EyeGaze"]
        
        # Calculating ideal (best, worst) values
        ideal_best_Emotion = df["EmotionNormalised"].max()
        ideal_worst_Emotion = df["EmotionNormalised"].min()
        ideal_best_Head = df["HeadTiltNormalised"].max()
        ideal_worst_Head = df["HeadTiltNormalised"].min()
        ideal_best_EyeGaze = df["EyeGazeNormalised"].max()
        ideal_worst_EyeGaze = df["EyeGazeNormalised"].min()
        
        # print(f"Emotion: {ideal_best_Emotion},{ideal_worst_Emotion}")
        # print(f"Head: {ideal_best_Head},{ideal_worst_Head}")
        # print(f"Eye: {ideal_best_EyeGaze},{ideal_worst_EyeGaze}")
        
        df['a_Vi+'] = df["EmotionNormalised"].max()
        df['a_Vi-'] = df["EmotionNormalised"].min()
        df['b_Vi+'] = df["HeadTiltNormalised"].max()
        df['b_Vi-'] = df["HeadTiltNormalised"].min()
        df['c_Vi+'] = df["EyeGazeNormalised"].max()
        df['c_Vi-'] = df["EyeGazeNormalised"].min()
        
        # Euclidean distance from ideal best (Si+) and ideal worst (Si-)
        def euclidean_distance(a,b,c):
            return ((a-ideal_best_Emotion)**2+(b-ideal_best_Head)**2+(c-ideal_best_EyeGaze)**2)**0.5
        
        df['Si+'] = df.apply(lambda row: ((row['EmotionNormalised']-ideal_best_Emotion)**2+(row['HeadTiltNormalised']-ideal_best_Head)**2+(row['EyeGazeNormalised']-ideal_best_EyeGaze)**2)**0.5,axis=1)
        df['Si-'] = df.apply(lambda row: ((row['EmotionNormalised']-ideal_worst_Emotion)**2+(row['HeadTiltNormalised']-ideal_worst_Head)**2+(row['EyeGazeNormalised']-ideal_worst_EyeGaze)**2)**0.5,axis=1)
        
        # Calculate performance score (Pi)
        def score(a,b):
            return (b/(a+b))
        
        df['Pi'] = df.apply(lambda row: row['Si-']/(row['Si+']+row['Si-']),axis = 1)
        def classifier(x):
            if x<0.25:
                return 0
            elif x>=0.25 and x<0.5:
                return 1
            elif x>=0.5 and x<0.75:
                return 2
            else:
                return 3

        # df['Engaged/Disengaged'] = df['Pi_1'].apply(lambda x: 'e' if x>=0.5 else 'ne')
        df['Engaged/Disengaged'] = df['Pi'].apply(classifier)
        
        df['Rank'] = df['Pi'].rank(method='max',ascending=True)
    
        # df_all=df_all.append(df)
        df_all = pd.concat([df_all,df])
        
        conc_indices.append(df['Pi'].sum())

        
    
    return df_all,conc_indices,times,emotions, head_tilt,eyes

In [ ]:
num_frames=17*3
# num_frames = 10
# conc_indices_mtcnn,times_mtcnn,emotions_mtcnn,hp_mtcnn, eyes_mtcnn = multimodal_backend("mtcnn", num_frames)
df_oc, conc_indices_oc,times_oc,emotions_oc,hp_oc,eyes_oc = multimodal_backend("opencv",num_frames)
# df_oc.to_csv('df_ofc.csv')
# conc_indices_dl,times_dl,emotions_dl = emotion_backend("dlib")
# conc_indices_mp,times_mp,emotions_mp, hp_mp, eyes_mp = multimodal_backend("mediapipe",num_frames)
# conc_indices_rf,times_rf,emotions_rf = emotion_backend("retinafce")
# conc_indices_oc,times_oc,emotions_oc = multimodal_backend("ArcFace")

In [ ]:
df_oc

In [ ]:
# box_color = 

box = pd.read_csv('df_ofc.csv')

# if box['Pi'] < 0.2:
#     box_color = (255,0,0)
# elif box['Pi'] >= 0.2 and box['Pi'] < 0.4:
#     box_color = (255,167,0)
# elif box['Pi'] >= 0.4 and box['Pi'] < 0.6:
#     box_color = (255,244,0)
# elif box['Pi'] >= 0.6 and box['Pi'] < 0.8:
#     box_color = (163,255,0)
# else:
#     box_color = (44,186,0)


def box_color(a):
    if a < 0.2:
        return (0,0,255)
    elif a >= 0.2 and a < 0.4:
        return (0,167,255)
    elif a >= 0.4 and a < 0.6:
        return (0,244,255)
    elif a >= 0.6 and a < 0.8:
        return (0,255,163)
    else:
        return (0,186,44)
    
frame_names = box['Frame'].unique()

# print(len())

print(frame_names, box_color)
for k,f in enumerate(frame_names):
    img_this_context = cv2.imread(f)
    for i, val in box.iterrows():
#     print(i, j['Frame'])
        if val['Frame']==f:
            b = box_color(val['Pi'])
            cv2.rectangle(img_this_context, (val['x']-7, val['y']-7), (val['x'] + val['w']+7, val['y'] + val['h']+7), b, 2)
            # cv2.putText(img_this_context,f"{val['Face']} {val['Emotion']} {val['HeadTilt']} {val['EyeGaze']}", (val['x'], val['y']), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            cv2.putText(img_this_context,f"{val['Face']} {val['Emotion']} {val['HeadTilt']} {val['EyeGaze']}", (val['x'], val['y']), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 255), 1)
    name=f'./processed_3rd/frame{k}.jpg'
    # print(name,frame)
    try:
        print(conc_indices_oc[k])
        cv2.putText(img_this_context,f"ACI: {n_conc_indices_oc[k]*100:.2f}%", (1500, 100), cv2.FONT_HERSHEY_DUPLEX, 2,b, 1)
    except:
        break
    # cv2.putText(img_this_context,f"{conc_indices_oc[k]}", (1880, 1050), cv2.FONT_HERSHEY_SIMPLEX, 2,(255, 255, 255), 1)
    cv2.imwrite(name,img_this_context)

In [ ]:
# old output

# ['./data_3rd/frame1_1798.jpg' './data_3rd/frame2_1799.jpg'
#  './data_3rd/frame3_1800.jpg' './data_3rd/frame4_3598.jpg'
#  './data_3rd/frame5_3599.jpg' './data_3rd/frame6_3600.jpg'
#  './data_3rd/frame7_5398.jpg' './data_3rd/frame8_5399.jpg'
#  './data_3rd/frame9_5400.jpg' './data_3rd/frame10_7198.jpg'
#  './data_3rd/frame11_7199.jpg' './data_3rd/frame12_7200.jpg'
#  './data_3rd/frame13_8998.jpg' './data_3rd/frame14_8999.jpg'
#  './data_3rd/frame15_9000.jpg' './data_3rd/frame16_10798.jpg'
#  './data_3rd/frame17_10799.jpg' './data_3rd/frame18_10800.jpg'
#  './data_3rd/frame19_12598.jpg' './data_3rd/frame20_12599.jpg'
#  './data_3rd/frame21_12600.jpg' './data_3rd/frame22_14398.jpg'
#  './data_3rd/frame23_14399.jpg' './data_3rd/frame24_14400.jpg'
#  './data_3rd/frame25_16198.jpg' './data_3rd/frame26_16199.jpg'
#  './data_3rd/frame27_16200.jpg' './data_3rd/frame28_17998.jpg'
#  './data_3rd/frame29_17999.jpg' './data_3rd/frame30_18000.jpg'
#  './data_3rd/frame31_19798.jpg' './data_3rd/frame32_19799.jpg'
#  './data_3rd/frame33_19800.jpg' './data_3rd/frame34_21598.jpg'
#  './data_3rd/frame35_21599.jpg' './data_3rd/frame36_21600.jpg'
#  './data_3rd/frame37_23398.jpg' './data_3rd/frame38_23399.jpg'
#  './data_3rd/frame39_23400.jpg' './data_3rd/frame40_25198.jpg'
#  './data_3rd/frame41_25199.jpg' './data_3rd/frame42_25200.jpg'
#  './data_3rd/frame43_26998.jpg' './data_3rd/frame44_26999.jpg'
#  './data_3rd/frame45_27000.jpg' './data_3rd/frame46_28798.jpg'
#  './data_3rd/frame47_28799.jpg' './data_3rd/frame48_28800.jpg'
#  './data_3rd/frame49_30598.jpg' './data_3rd/frame50_30599.jpg'
#  './data_3rd/frame51_30600.jpg']

In [ ]:
print(conc_indices_oc,len(conc_indices_oc))
# print(times)

In [ ]:
import statistics
def calc_normalized_ci(data):
    #count faces in each frame 
    d1 = data.groupby(['Frame']).size().reset_index(name='count faces')
    hm=statistics.harmonic_mean(d1['count faces'])
    print(hm)
    
    n_conc_indices=[min(1,item/hm) for item in conc_indices_oc]
    return n_conc_indices
          
n_conc_indices_oc=calc_normalized_ci(df_oc)
print(n_conc_indices_oc)

In [ ]:
s = pd.Series(times_oc[:17*3])
s.describe()

<h1>Average of 3fp per min</h1>

In [ ]:
# normalized
def conc_norm(ci_frames):
    norm_ci_25min=[]
    for i in range(0,len(ci_frames),3):
        norm_ci_25min.append(sum(ci_frames[i:i+3])/3)
    return norm_ci_25min

In [ ]:
# cn_mtcnn = conc_norm(conc_indices_mtcnn)
# cn_mp = conc_norm(conc_indices_mp)
cn_oc = conc_norm(n_conc_indices_oc)

In [ ]:

# importing packages 
import seaborn as sns 
import matplotlib.pyplot as plt 
  
# loading dataset 
  
# draw lineplot 

lp=sns.lineplot( data=cn_oc) 
lp.set(xlabel='Time (minutes)',ylabel='concentration_index', title='Concentration index during a lecture (Time Horizon ≈ 17 min)')
plt.grid()
plt.show()

In [ ]:
# importing packages 
import seaborn as sns 
import matplotlib.pyplot as plt 
sns.set(style="ticks")  
sns.color_palette("rocket")
# loading dataset 
  
# draw lineplot 
lp=sns.lineplot( data=cn_mtcnn)
sns.lineplot( data=cn_mp)
sns.lineplot( data=cn_oc)
lp.set(xlabel='Time (minutes)',ylabel='concentration_index', title='Concentration index during a lecture (Time Horizon ≈ 17 min)')
plt.legend(title= "Deepface Backend", labels = ["MTCNN", "Mediapipe", "OpenCV"], loc="upper right")
plt.grid()
plt.show()

In [ ]:
import pandas as pd
s = pd.Series(norm_ci_25min[:17])
s.describe()

In [ ]:
path = r'img.jpg'

# Reading an image in default mode
image = cv2.imread(path)
window_name = 'image'
cv2.imshow(window_name, image)

# waits for user to press any key
# (this is necessary to avoid Python kernel form crashing)
cv2.waitKey(0)

# closing all open windows
cv2.destroyAllWindows()

In [ ]:

mc= {"happy":0, "sad":0,"neutral":0,"angry":0,"disgust":0,"surprise":0,"fear":0}
em_all=["happy", "sad","neutral","angry","disgust","surprise","fear"]
for e in emotions_oc:
    for em in em_all:
        mc[em]+=e.get(em,0)
print(mc)

# Frame to video

In [ ]:
import cv2
import os

image_folder = 'processed_3rd'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()